In [2]:
import pandas as pd
from Bio.Align import Applications as AlignApp

In [3]:
data = pd.read_csv("data/processed_data/AMP/PeptideMarkerMIC.csv")

In [49]:
TheSmallest = data.drop_duplicates(["Target", "Peptide"]).groupby("Target")[["Target", "Marker", "Peptide", "Log2MIC"]].apply(lambda x: x.nsmallest(10, columns=["Log2MIC"]))
peptides = TheSmallest[["Peptide"]].drop_duplicates()
peptides.index = [i for i in range(peptides.shape[0])]
peptides['PepID'] = [f'pep.{i}' for i in range(peptides.shape[0])]
TheSmallest.index = [i for i in range(TheSmallest.shape[0])]
TheSmallest = TheSmallest.merge(peptides, on='Peptide')

In [54]:
file = open('data/processed_data/Baseline/peptides.fasta', 'w')
for pair in peptides.values.tolist():
    file.write(f">{pair[1]}\n")
    file.write(f"{pair[0]}\n")
file.close()

In [ ]:
# Go to MAFFT

In [67]:
class Preprocessing():
    def __init__(self):
        return
        
    def ReadFASTA(self,filename,as_pd=True):
        if filename.split(".")[-1] not in ["fasta","fna","fa"]:
            raise ValueError('Invalid file format. Expected formats are ["fasta","fna","fa"].')
        file_handle = open(filename,"r")
        seqs = []
        seqid = []
        tmp_seq = ""
        for line in file_handle:
            if (line[0] == ">"):
                if tmp_seq != "":
                    seqs.append(tmp_seq)
                seqid.append(line.split("\n")[0][1:])
                tmp_seq = ""
            else:
                tmp_seq+=line.split("\n")[0]
        seqs.append(tmp_seq)
        file_handle.close()
        if as_pd:
            fasta = {}
            for i in range(len(seqs)):
                fasta[seqid[i]] = seqs[i]
            return pd.DataFrame(fasta,index=["AlnPep"]).transpose()
        else:
            return seqs, seqid

In [72]:
pp = Preprocessing()
aln = pp.ReadFASTA('data/processed_data/Baseline/aln_peptides.fasta')
aln['PepID'] = aln.index
aln.index = [i for i in range(aln.shape[0])]

In [78]:
TheSmallest.merge(aln, on="PepID")[["Target", "Marker", "AlnPep"]]

,Target,Marker,AlnPep
0,Acholeplasma_laidlawii,AGAGUUUGAUCCUGGCUCAGGAUGAACGCUGGCGGCGUGCCUAAUA...,----------------------------------------------...
1,Mycoplasma_gallisepticum,AGAGUUUGAUCCUGGCUCAGGAUUAACGCUGGCGGCAUGCCUAAUA...,----------------------------------------------...
2,Mycoplasma_mycoides,AGAGUUUGAUCCUGGCUCAGGAUAAACGCUGGCGGCAUGCCUAAUA...,----------------------------------------------...
3,Spiroplasma_citri,UUUUUAAUGAGAGUUUGAUCCUGGCUCAGGAUGAACGCUGGCGGCA...,----------------------------------------------...
4,Spiroplasma_floricola,UGGAGAGUUUGAUCCUGGCUCAGGAUGAACGCUGGCGGCAUGCUUA...,----------------------------------------------...
...,...,...,...
1763,Yersinia_ruckeri,CCUUUGGCGGCAGGCUACACAUGCAGUCGAGCGGCAGCGGAAAGUA...,----------------------------------------------...
1764,Zobellia_galactanivorans,CAACGAUGAAGAGUUUGAUCCUGGCUCAGGAUGAACGCUAGCGGCA...,-----------------------------------------GCA--...
1765,Zymomonas_mobilis,UUGAUCCUGGCUCAGAACGAACGCUGGCGGCAUGCUUAACACAUGC...,----------------------------------------------...
1766,Zymomonas_mobilis,UUGAUCCUGGCUCAGAACGAACGCUGGCGGCAUGCUUAACACAUGC...,----------------------------------------------...


In [79]:
TheSmallest.to_csv('data/processed_data/Baseline/basedata.csv', index=False)